In [1]:
import master
from master.app import app

from master import models
from master.models import Detection, Identity


In [6]:
with app.app_context():
    ids = Identity.query.all()

print(ids)

[Identity('Esra Görkey', '{}'), Identity('Batuhan Görkey', 'None'), Identity('Işıl Görkey', 'None')]


In [7]:
import cv2
import numpy as np
from PIL import Image
import os

face_samples = []
ids = []

with app.app_context():
    dets = Detection.query.all()
    for det in dets:
        if det.identity:
            path = os.path.join("master/uploads/", det.filepath)
            PIL_img = Image.open(path).convert("L")
            img_numpy = np.array(PIL_img, "uint8")
            face_samples.append(img_numpy)
            ids.append(det.identity.id)

In [9]:
os.makedirs("trainer", exist_ok=True)

recognizer = cv2.face.LBPHFaceRecognizer.create()
recognizer.train(face_samples, np.array(ids))

recognizer.write("trainer/trainer.yml")

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os

# Path for face image database
path = "master/uploads/"
recognizer = cv2.face.LBPHFaceRecognizer.create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")


# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    faceSamples = []
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert("L")
        img_numpy = np.array(PIL_img, "uint8")
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for x, y, w, h in faces:
            faceSamples.append(img_numpy[y : y + h, x : x + w])
            ids.append(id)
    return faceSamples, ids


print("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces, ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))
# Save the model into trainer/trainer.yml
recognizer.write("trainer/trainer.yml")
# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))